# Training a classifier with Pytorch 
## First tutorial of pytorch 
### January 28th 2020

In [12]:
import sys
import os
import torch
#Data loaders:
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
#Define CNN:
import torch.nn as nn
import torch.nn.functional as F
#Optimizer 
import torch.optim as optim

In [2]:
print(sys.executable)

/home/razielar/bin/miniconda3/envs/pytorch/bin/python


In [14]:
print(os.getcwd())

/home/razielar/Documents/git_rep/Deep-Learning-AI-Barcelona-2018/Pytorch


## We are going to use CIFAR10 

In [16]:
#Normalization of images: range[0,1]. We want to transform them into tensors of normalized range [-1,1]

### --- Train:
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5,0.5,0.5))])

trainset=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader=torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)


100.0%

Extracting ./data/cifar-10-python.tar.gz to ./data


In [17]:
### --- Test: 
testset=torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

testloader=torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes=('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
